## Analysis of type 2 diabetes (T2D)

This notebook applies the `Get_Job_Script.ipynb` to automatically generate the sbatch scripts to run in Yale's cluster. The end result is to apply [various LMM workflows](https://github.com/statgenetics/UKBB_GWAS_dev/tree/master/workflow) to perform association analysis in the T2D trait, do clumping analysis and extract associated regions.

## File paths on Yale cluster

- Genotype files in PLINK format:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv`
- Genotype files in bgen format:
`SAY/dbgapstg/scratch/UKBiobank/genotype_files/ukb39554_imputeddataset/`
- Summary stats for imputed variants BOLT-LMM:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data`
- Summary stats for inputed variants FastGWA:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data`
- Phenotype files:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis`
- Relationship file:
`/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620`


## 07/01/20 analysis

On the cluster, open up this notebook using the JupyterLab server you set up via the ssh channel, then run the following cells,

## Bash variables for workflow configuration

In [1]:
# Common variables
tpl_file=../farnam.yml
bfile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.bed
sampleFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb32285_imputedindiv.sample
bgenFile=`echo /gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/ukb39554_imputeddataset/ukb_imp_chr{1..22}_v3.bgen`
unrelated_samples=/gpfs/gibbs/pi/dewan/data/UKBiobank/genotype_files/pleiotropy_geneticfiles/unrelated_n307259/UKB_unrelatedcauc_phenotypes_asthmat2dbmiwaisthip_agesex_waisthipratio_040620
formatFile_fastgwa=~/project/UKBB_GWAS_dev/data/fastGWA_template.yml
formatFile_bolt=~/project/UKBB_GWAS_dev/data/boltlmm_template.yml
formatFile_saige=~/project/UKBB_GWAS_dev/data/saige_template.yml
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
container_marp=/gpfs/gibbs/pi/dewan/data/UKBiobank/marp.sif
# LMM directories
lmm_dir_fastgwa=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D
lmm_dir_bolt=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/T2D
lmm_dir_saige=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/SAIGE_results/results_imputed_data/T2D
lmm_sos=../workflow/LMM.ipynb
lmm_sbatch_fastgwa=../output/$(date +"%Y-%m-%d")_T2D-fastgwa.sbatch
lmm_sbatch_bolt=../output/$(date +"%Y-%m-%d")_T2D-bolt.sbatch
lmm_sbatch_saige=../output/$(date +"%Y-%m-%d")_T2D-saige.sbatch
phenoFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720
## LMM variables 
covarFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/phenotype_files/pleiotropy_R01/phenotypesforanalysis/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720
LDscoresFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/LDSCORE.1000G_EUR.tab.gz
geneticMapFile=/gpfs/gibbs/pi/dewan/data/UKBiobank/genetic_map_hg19_withX.txt.gz
phenoCol=T2D
covarCol=SEX
covarMaxLevels=10
qCovarCol=AGE
numThreads=20
bgenMinMAF=0.001
bgenMinINFO=0.8
lmm_job_size=1
### Specific to SAIGE
bgenMinMAC=4
trait_type=binary
loco=TRUE
sampleCol=IID
# LD clumping directories
clumping_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/T2D
clumping_sos=../workflow/LD_Clumping.ipynb
clumping_sbatch=../output/$(date +"%Y-%m-%d")_T2D_ldclumping.sbatch
## LD clumping variables
# For sumtastsFiles if more than one provide each path
bfile_ref=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/UKB_Caucasians_phenotypeindepqc120319_updated020720removedwithdrawnindiv.1210.ref_geno.bed
# Changes dependending upon which traits are analyzed
# In this case T2D and INT-WHR
sumstatsFiles="/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz \
              /gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WHR/ukb_imp_v3.UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020.BoltLMM.snp_stats.all_chr.gz"
ld_sample_size=1210
clump_field=P
clump_p1=5e-08
clump_p2=1
clump_r2=0.2
clump_kb=2000
clump_annotate=BP
numThreads=20
clump_job_size=1
# Region extraction directories
extract_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/region_extraction/T2D
extract_sos=~/project/bioworkflows/GWAS/Region_Extraction.ipynb
extract_sbatch=../output/$(date +"%Y-%m-%d")_T2D-region.sbatch
## Region extraction variables
region_file=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/LD_clumping/asthma_INT-WHR_T2D/*.clumped_region
geno_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/UKBB_bgenfilepath.txt
sumstats_path=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz
extract_job_size=10

## BoltLMM job

In [ ]:
lmm_args="""boltlmm
    --cwd $lmm_dir_bolt 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_bolt 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_bolt \
    --args "$lmm_args"

## FastGWA job

In [ ]:
lmm_args="""fastGWA
    --cwd $lmm_dir_fastgwa 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_fastgwa 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_fastgwa \
    --args "$lmm_args"


## SAIGE job

In [ ]:
lmm_args="""SAIGE
    --cwd $lmm_dir_saige 
    --bfile $bfile 
    --sampleFile $sampleFile
    --bgenFile $bgenFile 
    --phenoFile $phenoFile 
    --formatFile $formatFile_saige 
    --covarFile $covarFile 
    --LDscoresFile $LDscoresFile 
    --geneticMapFile $geneticMapFile 
    --phenoCol $phenoCol 
    --covarCol $covarCol 
    --covarMaxLevels $covarMaxLevels 
    --qCovarCol $qCovarCol 
    --numThreads $numThreads 
    --bgenMinMAF $bgenMinMAF 
    --bgenMinINFO $bgenMinINFO 
    --bgenMinMAC $bgenMinMAC
    --trait_type $trait_type
    --loco $loco
    --sampleCol $sampleCol
    --job_size $lmm_job_size
    --ylim $ylim
    --container_lmm $container_lmm
    --container_marp $container_marp
"""

sos run ../workflow/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $lmm_sos \
    --to-script $lmm_sbatch_saige \
    --args "$lmm_args"

## LD clumping job

In [ ]:
clumping_args="""default 
    --cwd $clumping_dir 
    --bfile_ref $bfile_ref 
    --bgenFile $bgenFile
    --sampleFile $sampleFile 
    --sumstatsFiles $sumstatsFiles 
    --unrelated_samples $unrelated_samples 
    --ld_sample_size $ld_sample_size 
    --clump_field $clump_field
    --clump_p1 $clump_p1 
    --clump_p2 $clump_p2 
    --clump_r2 $clump_r2 
    --clump_kb $clump_kb 
    --clump_annotate $clump_annotate 
    --numThreads $numThreads 
    --job_size $clump_job_size
"""

sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $clumping_sos \
    --to-script $clumping_sbatch \
    --args "$clumping_args"

## Region extraction job

In [4]:
extract_args="""default
    --cwd $extract_dir
    --region-file $region_file
    --pheno-path $phenoFile
    --geno-path $geno_path
    --bgen-sample-path $sampleFile
    --sumstats-path $sumstats_path
    --format-config-path $formatFile_fastgwa
    --unrelated-samples $unrelated_samples
    --job-size $extract_job_size
"""

sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb dewan \
    --template-file $tpl_file \
    --workflow-file $extract_sos \
    --to-script $extract_sbatch \
    --args "$extract_args"

INFO: Running dewan: Configuration for Yale `pi_dewan` partition cluster
INFO: dewan is completed.
INFO: dewan output:   ../output/2021-05-03_T2D-region.sbatch
INFO: Workflow dewan (ID=wf6e25133daf5176d) is executed successfully with 1 completed step.



## Hudson Plots

### T2D vs INT-BMI

In [ ]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots/pleiotropy/T2D_INT-BMI
hudson_sbatch=../output/$(date +"%Y-%m-%d")_T2D_vs_INT-BMI_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D_091120/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz
sumstats_2=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-BMI/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-BMI_withagesex_041720_rankNorm_BMI.boltlmm.snp_stats.gz
toptitle="T2D_fastGWA"
bottomtitle="INT-BMI_boltlmm"
highlight_p_top=0
highlight_p_bottom=0
pval_filter=5e-06
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
#highlight_snp=
annotate_snp=0
phenocol1='T2D'
phenocol2='INT-BMI'

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
    --annotate_snp $annotate_snp
    --phenocol1 $phenocol1
    --phenocol2 $phenocol2
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

### T2D vs INT-WHR

In [ ]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots/pleiotropy/T2D_INT-WHR
hudson_sbatch=../output/$(date +"%Y-%m-%d")_T2D_vs_INT-WHR_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D_091120/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz
sumstats_2=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WHR/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WHR_withagesex_042020_rankNorm_WHR.boltlmm.snp_stats.gz
toptitle="T2D_fastGWA"
bottomtitle="INT-WHR_boltlmm"
highlight_p_top=0
highlight_p_bottom=0
pval_filter=5e-06
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
#highlight_snp=
annotate_snp=0
phenocol1='T2D'
phenocol2='INT-WHR'

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
    --annotate_snp $annotate_snp
    --phenocol1 $phenocol1
    --phenocol2 $phenocol2
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"

### T2D vs INT-WAIST

In [ ]:
tpl_file=../farnam.yml
hudson_sos=~/project/bioworkflows/GWAS/Hudson_plot.ipynb
hudson_dir=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/hudson_plots/pleiotropy/T2D_INT-WAIST
hudson_sbatch=../output/$(date +"%Y-%m-%d")_T2D_vs_INT-WAIST_hudson.sbatch
sumstats_1=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D_091120/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz
sumstats_2=/gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WAIST/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WAIST_withagesex_042020_rankNorm_WAIST.boltlmm.snp_stats.gz
toptitle="T2D_fastGWA"
bottomtitle="INT-WAIST_boltlmm"
highlight_p_top=0
highlight_p_bottom=0
pval_filter=5e-06
job_size=1
container_lmm=/gpfs/gibbs/pi/dewan/data/UKBiobank/lmm.sif
#highlight_snp=
annotate_snp=0
phenocol1='T2D'
phenocol2='INT-WAIST'

hudson_args="""hudson
    --cwd $hudson_dir
    --sumstats_1 $sumstats_1
    --sumstats_2 $sumstats_2
    --toptitle $toptitle
    --bottomtitle $bottomtitle
    --job_size $job_size
    --highlight_p_top $highlight_p_top
    --highlight_p_bottom $highlight_p_bottom
    --pval_filter $pval_filter
    --annotate_snp $annotate_snp
    --phenocol1 $phenocol1
    --phenocol2 $phenocol2
    --container_lmm $container_lmm
"""
sos run ~/project/bioworkflows/admin/Get_Job_Script.ipynb farnam \
    --template-file $tpl_file \
    --workflow-file $hudson_sos \
    --to-script $hudson_sbatch \
    --args "$hudson_args"